<a href="https://colab.research.google.com/github/dowrave/RoadToImageSeg_GAN/blob/main/GAN_MNIST_HandWriting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

## 데이터는 MNIST를 사용함

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28 ,1).astype('float32') # train_images.shape[0] = 샘플 수
train_images = (train_images - 127.5) / 127.5 # [-1, 1]로 정규화

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [ ]:
# 데이터 배치 만들고 섞기
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Generator
- 랜덤 -> 이미지 생성을 위해 Conv2DTranspose(업샘플링)을 이용한다.
- 처음 Dense층은 노이즈를 Input으로 받아, `28*28*1`이 나오도록 업샘플링을 한다.
- tanh를 사용하는 마지막층을 제외한 나머지 각 층마다 활성함수로 LeakyReLU를 쓴다

In [ ]:
def make_generator_model():
  '''이 함수 구성은 공부할 게 좀 많겠다'''
  model = tf.keras.Sequential()
  model.add(layers.Dense(7*7*256, use_bias = False, input_shape = (100, ))) # 7*7*256?
  model.add(layers.BatchNormalization()) # 얘 역할
  model.add(layers.LeakyReLU()) # 얘 역할

  model.add(layers.Reshape((7, 7, 256)))
  assert model.output_shape == (None, 7, 7, 256) # assert : 이 조건이 올바름을 개발자가 보장하며, 거짓이라면 AssertionError를 반환한다
                                                 # 오류 메시지를 작성하고 싶다면 assert(조건), '오류 메시지'
                                                 # BATCH_SIZE가 NONE임에 유의

  model.add(layers.Conv2DTranspose(128, (5, 5), strides = (1, 1), padding = 'same', use_bias = False))
  assert model.output_shape == (None, 7, 7, 128)
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(64, (5, 5), strides = (2, 2), padding = 'same', use_bias = False))
  assert model.output_shape == (None, 14, 14, 64)
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(1, (5, 5), strides = (2, 2), padding = 'same', use_bias = False, activation = 'tanh'))
  assert model.output_shape == (None, 28, 28 ,1)

  return model

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training = False) # ?

plt.imshow(generated_image[0, :, :, 0], cmap = 'gray')

## Discriminator
- CNN 기반의 이미지 분류기

In [ ]:
def make_discriminator_model():
  model = tf.keras.Sequential()
  model.add(layers.Conv2D(64, (5, 5), strides = (2, 2), padding = 'same', input_shape = [28, 28, 1]))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(128, (5, 5), strides = (2, 2), padding = 'same'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Flatten())
  model.add(layers.Dense(1)) # 이진 분류 : 진짜 이미지는 양수, 가짜 이미지는 음수

  return model

In [ ]:
discriminator = make_discriminator_model()
decision  = discriminator(generated_image)
print(decision)

## 손실함수, 옵티마이저 정의

In [ ]:
# 크로스엔트로피 손실함수를 계산하는 헬퍼 함수를 반환함
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

### Discriminator Loss Function
- 진짜 이미지와 1로 이루어진 행렬을 비교
- 가짜 이미지와 0으로 이루어진 행렬을 비교
- 이 두 크로스엔트로피 값을 합침

In [ ]:
def discrimniator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output) # xx_like(array) : array의 shape를 가졌는데 xx로 채움
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss


## Generator Loss Function
- 가짜 이미지와 진짜 행렬(1)을 비교하기

In [ ]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# 두 손실함수의 옵티마이저는 다름 : 별도로 훈련되니까 별도의 변수로 지정해준다
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### 체크포인트
- 훈련이 방해되는 경우 쓰이는 모델의 저장 방법과 복구 방법

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                 discrimniator_optimizer = discriminator_optimizer,
                                 generator = generator,
                                 discriminator = discriminator)

In [ ]:
# 훈련 루프 정의하기
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16 

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function
def train_step(images):
  noise = tf.random.normal([BATCH_SIZE, noise_dim])

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(noise, training = True)
    
    real_output = discriminator(images, training = True)
    fake_output = discriminator(generated_images, training = True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discrimniator_loss(real_output, fake_output)

  gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # # GIF 이미지 생성
    display.clear_output(wait = True)
    generate_and_save_images(generator, epoch + 1, seed)

    # 15에포크마다 모델 저장
    if (epoch + 1) % 15 == 0 :
      checkpoint.save(file_prefix = checkpoint_prefix)

    print(f'Time for epoch {epoch + 1} is {time.time() - start} sec')

    # 마지막 에포크가 끝난 후 생성
  display.clear_output(wait = True)
  generate_and_save_images(generator, epochs, seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Training = False일 때 Inference 모드로 실행됨(복습 : 모델엔 훈련 모드와 추론 모드 2가지가 있음)
  predictions = model(test_input, training = False)

  fig = plt.figure(figsize = (4, 4))

  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i+ 1)
    plt.imshow(predictions[i, : , :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

## 모델 훈련
- train() 메서드를 호출
- 중요 : 생성자 - 감별자가 서로를 제압하지 않는 것이 중요
  - 학습률이 비슷하다면 한쪽이 우세해진다.


In [ ]:
%%time
train(train_dataset, EPOCHS)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## GIF 생성

In [ ]:
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode = "I") as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i, filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame 
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

  import IPython
  if IPython.version_info > (6,2,0, ''):
   display.Image(filename = anim_file)

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(anim_file)